<a href="https://colab.research.google.com/github/ruilejin/LLM-review-sentiment/blob/main/Cross_validation_fold_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import numpy as np
import pandas as pd
import torch
from sklearn.model_selection import KFold

<frozen importlib._bootstrap>:219: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [ ]:
def kfold_data(k,dataframe):
    X = list(dataframe['review_content'])
    y1 = list(dataframe['review_type'])
    dummies=pd.get_dummies(y1)
    y=list(dummies['Fresh']) # Fresh is 1, Rotten is 0.

    kf=KFold(n_splits=k, shuffle=True, random_state=1)

    for train_index, test_index in kf.split(X):
        X_train=[]
        X_test=[]
        y_train=[]
        y_test=[]
        for i in train_index:
            X_train.append(X[i])
            y_train.append(y[i])
        for i in test_index:
            X_test.append(X[i])
            y_test.append(y[i])
        yield X_train,y_train,X_test,y_test

In [ ]:
def prepare_kfold_datasets(k,dataframe):
    train_datasets=[]
    test_datasets=[]
    X_train_for_all=[]
    y_train_for_all=[]
    X_test_for_all=[]
    y_test_for_all=[]
    for X_train,y_train,X_test,y_test in kfold_data(k,dataframe):
        X_train_for_all.append(X_train)
        y_train_for_all.append(y_train)
        X_test_for_all.append(X_test)
        y_test_for_all.append(y_test)
    return X_train_for_all,y_train_for_all,X_test_for_all,y_test_for_all

In [ ]:
df=reviews_all=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/data/data_use.csv',sep=',')

In [ ]:
df

,rotten_tomatoes_link,review_type,review_content,genres
0,m/10009583-fade_to_black,Rotten,All credit to writer-director Oliver Parker fo...,Mystery & Suspense
1,m/10009583-fade_to_black,Rotten,On paper this must have looked a neat idea: a ...,Mystery & Suspense
2,m/10009583-fade_to_black,Rotten,"A retro pastiche that, like The Good German be...",Mystery & Suspense
3,m/10009583-fade_to_black,Rotten,"Like Black Magic itself, alas, this is destine...",Mystery & Suspense
4,m/10009583-fade_to_black,Fresh,"For all its flaws, it's a movie-lover's movie,...",Mystery & Suspense
...,...,...,...,...
91804,m/krampus,Fresh,"Krampus is undoubtedly slight, but it's also a...",Horror
91805,m/crimson_peak,Fresh,"Not only did I love every silly, gothic, gorge...",Horror
91806,m/needful_things,Fresh,any movie that has the devil laughing to ave m...,Horror
91807,m/the_clovehitch_killer,Fresh,The Clovehitch Killer is one of those films th...,Horror


In [ ]:
genre_moviesno=df.groupby(['genres']).count()

In [ ]:
genre_moviesno.sort_values(by=['rotten_tomatoes_link'],ascending=False)

,rotten_tomatoes_link,review_type,review_content
genres,,,
Comedy,20000,20000,20000
Drama,20000,20000,20000
Horror,18976,18976,18976
Action & Adventure,14148,14148,14148
Mystery & Suspense,13460,13460,13460
Documentary,5225,5225,5225


In [ ]:
genre=['Drama','Comedy','Documentary','Horror','Mystery & Suspense','Action & Adventure']
df0=df.loc[df['genres'].isin([genre[0]])]
df1=df.loc[df['genres'].isin([genre[1]])]
df2=df.loc[df['genres'].isin([genre[2]])]
df3=df.loc[df['genres'].isin([genre[3]])]
df4=df.loc[df['genres'].isin([genre[4]])]
df5=df.loc[df['genres'].isin([genre[5]])]

In [ ]:
X_train_drama,y_train_drama,X_test_drama,y_test_drama = prepare_kfold_datasets(5,df0)
X_train_comedy,y_train_comedy,X_test_comedy,y_test_comedy = prepare_kfold_datasets(5,df1)
X_train_documentary,y_train_documentary,X_test_documentary,y_test_documentary = prepare_kfold_datasets(5,df2)
X_train_horror,y_train_horror,X_test_horror,y_test_horror = prepare_kfold_datasets(5,df3)
X_train_mystery,y_train_mystery,X_test_mystery,y_test_mystery = prepare_kfold_datasets(5,df4)
X_train_action,y_train_action,X_test_action,y_test_action = prepare_kfold_datasets(5,df5)

In [ ]:
all=[]
X_train_all=[]
for i1,i2,i3,i4,i5,i6 in zip(X_train_drama,X_train_comedy,X_train_documentary,X_train_horror,X_train_mystery,X_train_action):
    all=i1+i2+i3+i4+i5+i6
    X_train_all.append(all)

In [ ]:
all=[]
y_train_all=[]
for i1,i2,i3,i4,i5,i6 in zip(y_train_drama,y_train_comedy,y_train_documentary,y_train_horror,y_train_mystery,y_train_action):
    all=i1+i2+i3+i4+i5+i6
    y_train_all.append(all)

In [ ]:
all=[]
X_test_all=[]
for i1,i2,i3,i4,i5,i6 in zip(X_test_drama,X_test_comedy,X_test_documentary,X_test_horror,X_test_mystery,X_test_action):
    all=i1+i2+i3+i4+i5+i6
    X_test_all.append(all)

In [ ]:
all=[]
y_test_all=[]
for i1,i2,i3,i4,i5,i6 in zip(y_test_drama,y_test_comedy,y_test_documentary,y_test_horror,y_test_mystery,y_test_action):
    all=i1+i2+i3+i4+i5+i6
    y_test_all.append(all)

Save Data by Genre

In [ ]:
X_train_drama_df=pd.DataFrame(data=X_train_drama)
X_train_drama_df.to_csv('/content/drive/MyDrive/Colab Notebooks/data/genre/X_train_drama.csv',index=False)

y_train_drama_df=pd.DataFrame(data=y_train_drama)
y_train_drama_df.to_csv('/content/drive/MyDrive/Colab Notebooks/data/genre/y_train_drama.csv',index=False)

X_test_drama_df=pd.DataFrame(data=X_test_drama)
X_test_drama_df.to_csv('/content/drive/MyDrive/Colab Notebooks/data/genre/X_test_drama.csv',index=False)

y_test_drama_df=pd.DataFrame(data=y_test_drama)
y_test_drama_df.to_csv('/content/drive/MyDrive/Colab Notebooks/data/genre/y_test_drama.csv',index=False)

In [ ]:
X_train_comedy_df=pd.DataFrame(data=X_train_comedy)
X_train_comedy_df.to_csv('/content/drive/MyDrive/Colab Notebooks/data/genre/X_train_comedy.csv',index=False)

y_train_comedy_df=pd.DataFrame(data=y_train_comedy)
y_train_comedy_df.to_csv('/content/drive/MyDrive/Colab Notebooks/data/genre/y_train_comedy.csv',index=False)

X_test_comedy_df=pd.DataFrame(data=X_test_comedy)
X_test_comedy_df.to_csv('/content/drive/MyDrive/Colab Notebooks/data/genre/X_test_comedy.csv',index=False)

y_test_comedy_df=pd.DataFrame(data=y_test_comedy)
y_test_comedy_df.to_csv('/content/drive/MyDrive/Colab Notebooks/data/genre/y_test_comedy.csv',index=False)

In [ ]:
X_train_documentary_df=pd.DataFrame(data=X_train_documentary)
X_train_documentary_df.to_csv('/content/drive/MyDrive/Colab Notebooks/data/genre/X_train_documentary.csv',index=False)

y_train_documentary_df=pd.DataFrame(data=y_train_documentary)
y_train_documentary_df.to_csv('/content/drive/MyDrive/Colab Notebooks/data/genre/y_train_documentary.csv',index=False)

X_test_documentary_df=pd.DataFrame(data=X_test_documentary)
X_test_documentary_df.to_csv('/content/drive/MyDrive/Colab Notebooks/data/genre/X_test_documentary.csv',index=False)

y_test_documentary_df=pd.DataFrame(data=y_test_documentary)
y_test_documentary_df.to_csv('/content/drive/MyDrive/Colab Notebooks/data/genre/y_test_documentary.csv',index=False)

In [ ]:
X_train_horror_df=pd.DataFrame(data=X_train_horror)
X_train_horror_df.to_csv('/content/drive/MyDrive/Colab Notebooks/data/genre/X_train_horror.csv',index=False)

y_train_horror_df=pd.DataFrame(data=y_train_horror)
y_train_horror_df.to_csv('/content/drive/MyDrive/Colab Notebooks/data/genre/y_train_horror.csv',index=False)

X_test_horror_df=pd.DataFrame(data=X_test_horror)
X_test_horror_df.to_csv('/content/drive/MyDrive/Colab Notebooks/data/genre/X_test_horror.csv',index=False)

y_test_horror_df=pd.DataFrame(data=y_test_horror)
y_test_horror_df.to_csv('/content/drive/MyDrive/Colab Notebooks/data/genre/y_test_horror.csv',index=False)

In [ ]:
X_train_mystery_df=pd.DataFrame(data=X_train_mystery)
X_train_mystery_df.to_csv('/content/drive/MyDrive/Colab Notebooks/data/genre/X_train_mystery.csv',index=False)

y_train_mystery_df=pd.DataFrame(data=y_train_mystery)
y_train_mystery_df.to_csv('/content/drive/MyDrive/Colab Notebooks/data/genre/y_train_mystery.csv',index=False)

X_test_mystery_df=pd.DataFrame(data=X_test_mystery)
X_test_mystery_df.to_csv('/content/drive/MyDrive/Colab Notebooks/data/genre/X_test_mystery.csv',index=False)

y_test_mystery_df=pd.DataFrame(data=y_test_mystery)
y_test_mystery_df.to_csv('/content/drive/MyDrive/Colab Notebooks/data/genre/y_test_mystery.csv',index=False)

In [ ]:
X_train_action_df=pd.DataFrame(data=X_train_action)
X_train_action_df.to_csv('/content/drive/MyDrive/Colab Notebooks/data/genre/X_train_action.csv',index=False)

y_train_action_df=pd.DataFrame(data=y_train_action)
y_train_action_df.to_csv('/content/drive/MyDrive/Colab Notebooks/data/genre/y_train_action.csv',index=False)

X_test_action_df=pd.DataFrame(data=X_test_action)
X_test_action_df.to_csv('/content/drive/MyDrive/Colab Notebooks/data/genre/X_test_action.csv',index=False)

y_test_action_df=pd.DataFrame(data=y_test_action)
y_test_action_df.to_csv('/content/drive/MyDrive/Colab Notebooks/data/genre/y_test_action.csv',index=False)

Save Data by All

In [ ]:
X_train_all_df=pd.DataFrame(data=X_train_all)
X_train_all_df.to_csv('/content/drive/MyDrive/Colab Notebooks/data/all/X_train_all.csv',index=False)

y_train_all_df=pd.DataFrame(data=y_train_all)
y_train_all_df.to_csv('/content/drive/MyDrive/Colab Notebooks/data/all/y_train_all.csv',index=False)

X_test_all_df=pd.DataFrame(data=X_test_all)
X_test_all_df.to_csv('/content/drive/MyDrive/Colab Notebooks/data/all/X_test_all.csv',index=False)

y_test_all_df=pd.DataFrame(data=y_test_all)
y_test_all_df.to_csv('/content/drive/MyDrive/Colab Notebooks/data/all/y_test_all.csv',index=False)